# Building constraints for calibration
In this notebook, we will take all the measurements we pulled from EXFOR and build `Constraint` and `Model` objects defined in `calibration.py` in this repo. These classes together allow us to precompute useful things (like kinematics and Coulomb scattering amplitudes) for model calculations for each measurement, so that wecan toss in a model and a bunch of parameters and quickly generate and ensemble of predictions for our data corpus. and calculate things like $\chi^2$ distributions and empirical coverages.

In [1]:
import pickle
from collections import OrderedDict

import numpy as np
from periodictable import elements
from tqdm import tqdm

In [2]:
from exfor_tools import curate, quantities, reaction

Using database version X4-2024-12-31 located in: /home/beyerk/db/exfor/unpack_exfor-2024/X4-2024-12-31


In [3]:
from jitr import data_dir, rmatrix
from jitr.optical_potentials import kduq, wlh
from jitr.utils import kinematics
from jitr.xs.elastic import DifferentialWorkspace

In [4]:
import elm

In [5]:
proton = (1, 1)
neutron = (1, 0)

## Read in the measurements

In [6]:
with open("nn_elastic_data.pkl", "rb") as f:
    nn_data = pickle.load(f)

In [7]:
with open("pp_elastic_data.pkl", "rb") as f:
    pp_data = pickle.load(f)

In [8]:
all_data = list(pp_data.values()) + list(nn_data.values())
all_data_by_entry, sys_uncertainties_by_entry = (
    curate.cross_reference_entry_systematic_err(all_data)
)

## Set up the Constraints
We will store everything in a single `dict` that maps from subentry to a `tuple` of (`ReactionConstraint`, `ElasticModel`)

In [9]:
constraints = {}

In [10]:
core_solver = rmatrix.Solver(40)
angles_vis = np.linspace(0.01, 180, 100)

for target, dataset in tqdm(pp_data.items()):
    if dataset.reaction not in constraints:
        constraints[dataset.reaction] = []

    for quantity, reaction_entries in dataset.data.items():
        for entry_id, entry in reaction_entries.entries.items():
            for m in entry.measurements:

                model = elm.ElasticModel(
                    quantity="dXS/dRuth",
                    reaction=dataset.reaction,
                    Elab=m.Einc,
                    angles_rad_constraint=m.x * np.pi / 180,
                    angles_rad_vis=angles_vis * np.pi / 180,
                    core_solver=core_solver,
                    lmax=30,
                )

                if quantity == "dXS/dRuth":
                    norm = None
                elif quantity == "dXS/dA":
                    norm = model.constraint_workspace.rutherford
                else:
                    assert False
                constraint = elm.calibration.ReactionConstraint(
                    quantity="dXS/dRuth", measurement=m, normalize=norm
                )

                constraints[dataset.reaction].append((constraint, model))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 22/22 [03:27<00:00,  9.44s/it]


In [12]:
for target, dataset in tqdm(nn_data.items()):
    if dataset.reaction not in constraints:
        constraints[dataset.reaction] = []

    for quantity, reaction_entries in dataset.data.items():
        for entry_id, entry in reaction_entries.entries.items():
            for m in entry.measurements:

                model = elm.ElasticModel(
                    quantity="dXS/dA",
                    reaction=dataset.reaction,
                    Elab=m.Einc,
                    angles_rad_constraint=m.x * np.pi / 180,
                    angles_rad_vis=angles_vis * np.pi / 180,
                    core_solver=core_solver,
                    lmax=30,
                )

                constraint = elm.calibration.ReactionConstraint(
                    quantity="dXS/dA", measurement=m
                )

                constraints[dataset.reaction].append((constraint, model))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 22/22 [01:17<00:00,  3.52s/it]


In [13]:
with open("constraints.pkl", 'wb') as f:
    pickle.dump(constraints, f)